#### CSC 180 Intelligent Systems 

#### Dr. Haiquan Chen, Dept of Computer Scicence

#### California State University, Sacramento




## Lab 3: Evaluating Neural Networks 


### Helpful Functions for Tensorflow (little gems)

The following functions will be used with TensorFlow to help preprocess the data.  They allow you to build the feature vector for a neural network. 

* Predictors/Inputs 
    * Fill any missing inputs with the median for that column.  Use **missing_median**.
    * Encode textual/categorical values with **encode_text_dummy**.
    * Encode numeric values with **encode_numeric_zscore**.
* Output
    * Discard rows with missing outputs.
    * Encode textual/categorical values with **encode_text_index**.
    * Do not encode output numeric values.
* Produce final feature vectors (x) and expected output (y) with **to_xy**.


In [1]:
from collections.abc import Sequence
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low


# Training with a Test Set with Early Stopping

**Overfitting** occurs when a neural network is trained to the point that it begins to memorize rather than generalize.  




### Split data into training and test using train_test_split

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

path = "./data/"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")

x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()

model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

# patience: number of epochs with no improvement after which training will be stopped

# The test set is checked during training to monitor progress for early stopping but is never used for gradient descent (model training)

model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor], verbose=2, epochs=1000)  


Epoch 1/1000


/Users/phatdoan/CSC180-Intelligent-Systems/CSC180-Intelligent-Systems/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 - 1s - 190ms/step - loss: 2.7606 - val_loss: 2.8724
Epoch 2/1000
4/4 - 0s - 10ms/step - loss: 2.6024 - val_loss: 2.7022
Epoch 3/1000
4/4 - 0s - 9ms/step - loss: 2.4494 - val_loss: 2.5391
Epoch 4/1000
4/4 - 0s - 10ms/step - loss: 2.3093 - val_loss: 2.3826
Epoch 5/1000
4/4 - 0s - 10ms/step - loss: 2.1702 - val_loss: 2.2349
Epoch 6/1000
4/4 - 0s - 10ms/step - loss: 2.0415 - val_loss: 2.0956
Epoch 7/1000
4/4 - 0s - 10ms/step - loss: 1.9244 - val_loss: 1.9654
Epoch 8/1000
4/4 - 0s - 10ms/step - loss: 1.8169 - val_loss: 1.8454
Epoch 9/1000
4/4 - 0s - 9ms/step - loss: 1.7120 - val_loss: 1.7378
Epoch 10/1000
4/4 - 0s - 10ms/step - loss: 1.6250 - val_loss: 1.6405
Epoch 11/1000
4/4 - 0s - 10ms/step - loss: 1.5464 - val_loss: 1.5529
Epoch 12/1000
4/4 - 0s - 9ms/step - loss: 1.4699 - val_loss: 1.4744
Epoch 13/1000
4/4 - 0s - 10ms/step - loss: 1.4058 - val_loss: 1.4053
Epoch 14/1000
4/4 - 0s - 9ms/step - loss: 1.3503 - val_loss: 1.3450
Epoch 15/1000
4/4 - 0s - 9ms/step - loss: 1.3026 - val_loss

Now that the neural network is trained, we can make predictions about the test set.  The following code predicts the type of iris for test set and displays the first five irises. 

In [3]:
pred = model.predict(x_test)
print(pred[0:5]) # print first five predictions

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[8.1315534e-03 8.8928092e-01 1.0258758e-01]
 [9.9404633e-01 5.9531876e-03 5.2129241e-07]
 [7.3495699e-05 6.8051480e-02 9.3187511e-01]
 [6.0051852e-03 7.2153276e-01 2.7246195e-01]
 [4.7076237e-03 9.1033322e-01 8.4959164e-02]]


Each line provides the probability that the iris is one of the 3 types of iris in the data set. 

## Saving Best Weights

It would be good idea to keep track of the most optimal weights during the entire training operation.  


An additional monitor, ModelCheckpoint,  is used and saves a copy of the neural network to **best_weights.hdf5** each time the validation score of the neural network improves.  

Once training is done, we just reload this file and we have the optimal training weights that were found.

In [4]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "./data/"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")
x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

checkpointer = ModelCheckpoint(filepath="dnn/best_weights.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)

model.load_weights('dnn/best_weights.hdf5') # load weights from best model

/Users/phatdoan/CSC180-Intelligent-Systems/CSC180-Intelligent-Systems/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=dnn/best_weights.hdf5

Note that the use of HDF5 (.h5 or .hdf5) files for saving entire Keras models is being deprecated.  So if you are using the lateset versions of TensorFlow, ```you may need to replace .hdf5 with .keras```


You might occasionally see this error:

```
OSError: Unable to create file (Unable to open file: name = 'dnn/best_weights.hdf5', errno = 22, error message = 'invalid argument', flags = 13, o_flags = 302)
```

Usually you can just run rerun the code and it goes away.  This is an unfortnuate result of saving a file each time the validation score improves (as described in the previous section).  If the errors improve two rapidly, you might try to save the file twice and get an error from these two saves overlapping.  For larger neural networks this will not be a problem because each training step will take longer, allowing for plenty of time for the previous save to complete.   

## Evaluating Classification Models

### (1) Calculate Classification Accuracy/Precision/Recall/F1-Score

By default, Keras will return the predicted probability for each class. We can change these prediction probabilities into the actual iris predicted with **argmax**.

In [ ]:
pred = model.predict(x_test)
pred

array([[4.62278072e-03, 8.71348143e-01, 1.24029100e-01],
       [9.98032272e-01, 1.96734793e-03, 3.99851984e-07],
       [2.72625257e-05, 1.96773782e-02, 9.80295360e-01],
       [5.12181176e-03, 8.19644332e-01, 1.75233886e-01],
       [3.26915109e-03, 8.95215034e-01, 1.01515815e-01],
       [9.95334685e-01, 4.66378499e-03, 1.49596485e-06],
       [1.74063779e-02, 9.65007126e-01, 1.75864995e-02],
       [8.44821043e-04, 2.11847365e-01, 7.87307799e-01],
       [2.99428217e-03, 4.88658190e-01, 5.08347571e-01],
       [7.23424042e-03, 9.70882595e-01, 2.18832027e-02],
       [1.58895308e-03, 3.46978843e-01, 6.51432216e-01],
       [9.94751513e-01, 5.24606975e-03, 2.47903699e-06],
       [9.98743236e-01, 1.25652738e-03, 1.86740110e-07],
       [9.94842231e-01, 5.15547162e-03, 2.32703633e-06],
       [9.98633444e-01, 1.36600388e-03, 4.84896020e-07],
       [4.39007021e-03, 8.94291997e-01, 1.01317957e-01],
       [1.14667280e-04, 2.84153763e-02, 9.71469939e-01],
       [7.99867045e-03, 9.54486

In [ ]:
pred = np.argmax(pred,axis=1) # raw probabilities to choose class (highest probability)
print(pred)

[1 0 2 1 1 0 1 2 2 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]


Now that we have the actual iris flower predicted, we can calculate the percent accuracy (how many were correctly classified).

In [ ]:
y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

Accuracy score: 0.9736842105263158


In [ ]:
score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

Precision score: 0.9757085020242916


In [ ]:
score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

Recall score: 0.9736842105263158


In [ ]:
score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))

F1 score: 1.0


In [ ]:
print(metrics.classification_report(y_true, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      0.91      0.95        11
           2       0.92      1.00      0.96        12

    accuracy                           0.97        38
   macro avg       0.97      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38



### (2) Calculate Classification Cross-Entropy Loss (Log Loss)  

Log loss is an error metric that is often used in place of accuracy for classification.  

Log loss allows for "partial credit". For example, a model might be used to classify A, B and C.  The correct answer might be A, however if the classification network chose B as having the highest probability, then accuracy gives the neural network no credit for this classification.  

However, with log loss, the probability of the correct answer is added to the score.  For example, the correct answer might be A, but if the neural network only predicted .4 probability of A being correct, then the value -log(.4) is added.

$$ logloss = -\frac{1}{N}\sum^N_{i=1}\sum^M_{j=1}y_{ij} \log(\hat{y}_{ij}) $$

The following code shows the logloss scores that correspond to the average probablity for the correct item. The **pred** column specifies the average robability for the correct class.  The **logloss** column specifies the log loss for that probability.


Calculating log loss

In [ ]:
# Generate predictions
pred = model.predict(x_test)

print("Numpy array of predictions")
print(pred[0:5])
print()
print("y_test:")
print(y_test[0:5])

score = metrics.log_loss(y_test, pred)
print("Log loss score: {}".format(score))

Numpy array of predictions
[[4.62278072e-03 8.71348143e-01 1.24029100e-01]
 [9.98032272e-01 1.96734793e-03 3.99851984e-07]
 [2.72625257e-05 1.96773782e-02 9.80295360e-01]
 [5.12181176e-03 8.19644332e-01 1.75233886e-01]
 [3.26915109e-03 8.95215034e-01 1.01515815e-01]]

y_test:
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
Log loss score: 0.09630766139659834


## Evaluating Regression Models

Regression results are evaluated differently than classification.  Consider the following code. 

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import metrics

path = "./data/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

cars = df['name']
df.drop('name',1,inplace=True)
missing_median(df, 'horsepower')

encode_text_dummy(df, 'origin')

x,y = to_xy(df,"mpg")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size=0.25, random_state=45)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10))
model.add(Dense(10))
model.add(Dense(10))

model.add(Dense(1))  # 1 output neuron 


model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.fit(x_train,y_train, validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)


Epoch 1/1000
10/10 - 0s - loss: 222095.3750 - val_loss: 806.3899
Epoch 2/1000
10/10 - 0s - loss: 27754.6953 - val_loss: 41621.0508
Epoch 3/1000
10/10 - 0s - loss: 17286.1191 - val_loss: 653.4756
Epoch 4/1000
10/10 - 0s - loss: 4482.3921 - val_loss: 5349.0269
Epoch 5/1000
10/10 - 0s - loss: 2418.5466 - val_loss: 778.1186
Epoch 6/1000
10/10 - 0s - loss: 1349.0989 - val_loss: 1095.7517
Epoch 7/1000
10/10 - 0s - loss: 674.2885 - val_loss: 647.5696
Epoch 8/1000
10/10 - 0s - loss: 682.3955 - val_loss: 508.4194
Epoch 9/1000
10/10 - 0s - loss: 525.0196 - val_loss: 559.2294
Epoch 10/1000
10/10 - 0s - loss: 536.6884 - val_loss: 495.9714
Epoch 11/1000
10/10 - 0s - loss: 517.0453 - val_loss: 489.7463
Epoch 12/1000
10/10 - 0s - loss: 508.8820 - val_loss: 489.0688
Epoch 13/1000
10/10 - 0s - loss: 500.3905 - val_loss: 479.7199
Epoch 14/1000
10/10 - 0s - loss: 500.0540 - val_loss: 468.1486
Epoch 15/1000
10/10 - 0s - loss: 497.1146 - val_loss: 473.7314
Epoch 16/1000
10/10 - 0s - loss: 473.4629 - val_lo

### Mean Square Error

The mean square error is the sum of the squared differences between the prediction ($\hat{y}$) and the expected ($y$).  MSE values are not of a particular unit.  If an MSE value has decreased for a model, that is good. Low MSE values are desired.

$ \text{MSE} = \frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2 $


In [ ]:
# Predict
pred = model.predict(x_test)

# Measure MSE error.  
score = metrics.mean_squared_error(pred,y_test)
print("Final score (MSE): {}".format(score))

Final score (MSE): 18.79427146911621


### Root Mean Square Error

The root mean square (RMSE) is essentially the square root of the MSE.  Because of this, the RMSE error is in the same units as the training data outcome. Low RMSE values are desired.

$ \text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2} $

In [ ]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 4.335236072540283


# Performance Improvement by Normalizing Features and Tuning Hyperparameters

There are many different settings that you can use for a neural network.  These can affect performance.  The following code changes some of these, beyond their default values:

* **activation:** relu, sigmoid, tanh
* **Layers and Neuron Counts**
* **optimizer:** adam, sgd, rmsprop, and [others](https://keras.io/optimizers/)

In [ ]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "./data/"
preprocess = True

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

# create feature vector
missing_median(df, 'horsepower')
encode_text_dummy(df, 'origin')
df.drop('name',1,inplace=True)

if preprocess:
    encode_numeric_zscore(df, 'horsepower')
    encode_numeric_zscore(df, 'weight')
    encode_numeric_zscore(df, 'cylinders')
    encode_numeric_zscore(df, 'displacement')
    encode_numeric_zscore(df, 'acceleration')
    encode_numeric_zscore(df, 'year')

# Encode to a 2D matrix for training
x,y = to_xy(df,'mpg')

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

model = Sequential()
model.add(Dense(100, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)


Epoch 1/1000
10/10 - 0s - loss: 605.9440 - val_loss: 556.0004
Epoch 2/1000
10/10 - 0s - loss: 566.8333 - val_loss: 507.1982
Epoch 3/1000
10/10 - 0s - loss: 505.1319 - val_loss: 429.8139
Epoch 4/1000
10/10 - 0s - loss: 409.0138 - val_loss: 316.9863
Epoch 5/1000
10/10 - 0s - loss: 273.1548 - val_loss: 177.2618
Epoch 6/1000
10/10 - 0s - loss: 125.7235 - val_loss: 61.3011
Epoch 7/1000
10/10 - 0s - loss: 38.0301 - val_loss: 39.0215
Epoch 8/1000
10/10 - 0s - loss: 36.1851 - val_loss: 35.2568
Epoch 9/1000
10/10 - 0s - loss: 26.0564 - val_loss: 23.3026
Epoch 10/1000
10/10 - 0s - loss: 21.4498 - val_loss: 19.8781
Epoch 11/1000
10/10 - 0s - loss: 18.4882 - val_loss: 16.6677
Epoch 12/1000
10/10 - 0s - loss: 16.2612 - val_loss: 14.6550
Epoch 13/1000
10/10 - 0s - loss: 14.9690 - val_loss: 12.8048
Epoch 14/1000
10/10 - 0s - loss: 13.7016 - val_loss: 11.0336
Epoch 15/1000
10/10 - 0s - loss: 12.8530 - val_loss: 9.9106
Epoch 16/1000
10/10 - 0s - loss: 12.1488 - val_loss: 8.9551
Epoch 17/1000
10/10 - 0s

In [ ]:
# Predict and measure RMSE
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

Score (RMSE): 2.166762590408325


In [5]:
# print out prediction
df_y = pd.DataFrame(y_test, columns=['ground_truth'])
df_pred = pd.DataFrame(pred, columns=['predicted'])
result = pd.concat([df_y, df_pred],axis=1)
result

ValueError: Shape of passed values is (38, 3), indices imply (38, 1)